파인튜닝을 위한 코드이다. 이미 존재하는 모델에 추가 데이터를 투입하여 파라미터를 업데이트를 위한 코드이다.

#**1. 구글 드라이브를 코랩에 연결한다.**
> 이는 추후 모델을 불러오고, 학습할 데이터를 불러오기 위해 필요한 과정이다. \
 따라서 이 코드를 실행하기 전에, 구글 드라이브에 모델과 토크나이저, 전처리된 데이터를 업로드 해야 한다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#**2. 파인튜닝을 위해 필요한 라이브러리를 불러온다.**

In [ ]:
!pip install transformers seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizerFast, ElectraConfig, ElectraForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

#**3. 파인튜닝을 위한 데이터셋을 불러온다.**
> ***데이터의 경로 입력/수정 필수!***

In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/2022_lesik_workspace/lesik/data/total+yejib_train.tsv', sep = '\t', keep_default_na=False)
df.head()

,text,label
0,1. 오이와 대파는 5cm길이로 채를 썬 후 대파는 찬물에 담가주세요. 청상추는 한...,O O CV_INGREDIENT O CV_INGREDIENT O O O O O O ...
1,2. 차돌박이는 키친타올에 올려 핏물을 제거해주세요. 볼에 양념 재료를 넣어 섞어주세요.,O O CV_INGREDIENT CV_INGREDIENT CV_INGREDIENT ...
2,"4. 흐르는 찬물에 중면을 여러 번 씻어 전분기를 없애고, 채반에 받쳐 물기를 빼주세요.",O O O O O O CV_INGREDIENT CV_INGREDIENT O O O ...
3,5. 달군 팬에 차돌박이를 소금간을 하여 노릇하게 구워주세요.,O O O O O O CV_INGREDIENT CV_INGREDIENT CV_ING...
4,6. 섞어둔 양념장에 중면을 넣고 고루 버무려주세요. 청상추를 넣어 가볍게 버무려주세요.,O O CV_STATE CV_STATE CV_STATE CV_SEASONING CV...


태그는 최근 모델과 동일해야 되므로 변경해서는 안된다.

In [ ]:
# Split labels based on whitespace and turn them into a list
arr_labels = set()
for lb in df.label:
    lb = lb.split()
    for ll in lb:
        if ll not in arr_labels:
            arr_labels.add(ll)

#말뭉치 데이터에 포함된 총 태그
unique_labels = {'OGG_EDUCATION', 'MT_ELEMENT', 'AFW_OTHER_PRODUCTS', 'MT_ROCK', 'TI_OTHERS', 'PS_NAME', 'CV_BUILDING_TYPE', 'AM_REPTILIA', 'OGG_FOOD', 'AF_MUSICAL_INSTRUMENT', 'AF_BUILDING', 'AFA_MUSIC', 'CV_SPORTS_INST', 'QT_ORDER', 'TM_COLOR', 'LCG_MOUNTAIN', 'QT_MAN_COUNT', 'PS_CHARACTER', 'AM_OTHERS', 'OGG_LIBRARY', 'TMM_DISEASE', 'OGG_MEDICINE', 'LCG_ISLAND', 'TI_MINUTE', 'MT_CHEMICAL', 'TM_CELL_TISSUE_ORGAN', 'QT_OTHERS', 'CV_TRIBE', 'QT_TEMPERATURE', 'PT_FLOWER', 'OGG_POLITICS', 'DT_WEEK', 'FD_ART', 'AM_AMPHIBIA', 'FD_MEDICINE', 'AF_CULTURAL_ASSET', 'AF_TRANSPORT', 'EV_SPORTS', 'LCG_CONTINENT', 'PT_TREE', 'TMI_SERVICE', 'AM_MAMMALIA', 'TM_SPORTS', 'CV_INGREDIENT', 'OGG_HOTEL', 'QT_PHONE', 'CV_LANGUAGE', 'CV_FUNDS', 'CV_CURRENCY', 'FD_OTHERS', 'LCG_RIVER', 'LCP_CAPITALCITY', 'LC_OTHERS', 'QT_SIZE', 'TM_CLIMATE', 'TM_SHAPE', 'CV_POLICY', 'EV_ACTIVITY', 'TR_ART', 'QT_ADDRESS', 'OGG_RELIGION', 'CV_POSITION', 'FD_HUMANITIES', 'CV_CULTURE', 'QT_SPORTS', 'QT_ALBUM', 'CV_ART', 'CV_FOOD', 'CV_LAW', 'OGG_MILITARY', 'DT_DAY', 'FD_SOCIAL_SCIENCE', 'LCP_PROVINCE', 'CV_CLOTHING', 'TI_HOUR', 'DT_DYNASTY', 'DT_SEASON', 'FD_SCIENCE', 'TMI_HW', 'OGG_SPORTS', 'TR_OTHERS', 'TM_DIRECTION', 'TMI_SITE', 'QT_LENGTH', 'MT_METAL', 'LCG_OCEAN', 'DT_OTHERS', 'LCP_COUNTY', 'TMIG_GENRE', 'OGG_ECONOMY', 'TMI_SW', 'CV_SPORTS_POSITION', 'AFA_DOCUMENT', 'PT_OTHERS', 'AFA_ART_CRAFT', 'EV_OTHERS', 'TMI_EMAIL', 'QT_PRICE', 'EV_FESTIVAL', 'TI_SECOND', 'CV_TAX', 'O', 'QT_VOLUME', 'AF_WEAPON', 'LCG_BAY', 'OGG_SCIENCE', 'PT_FRUIT', 'CV_OCCUPATION', 'QT_CHANNEL', 'OGG_ART', 'AM_INSECT', 'CV_FOOD_STYLE', 'QT_PERCENTAGE', 'OGG_LAW', 'TR_SCIENCE', 'CV_RELATION', 'AM_PART', 'QT_AGE', 'TMI_MODEL', 'AM_BIRD', 'OGG_OTHERS', 'CV_SPORTS', 'DT_YEAR', 'LCP_COUNTRY', 'AFA_VIDEO', 'DT_GEOAGE', 'TI_DURATION', 'AM_TYPE', 'CV_SEASONING', 'AM_FISH', 'CV_PRIZE', 'PS_PET', 'AFW_SERVICE_PRODUCTS', 'TMI_PROJECT', 'CV_DRINK', 'LC_SPACE', 'LCP_CITY', 'EV_WAR_REVOLUTION', 'AFA_PERFORMANCE', 'QT_SPEED', 'PT_GRASS', 'DT_MONTH', 'PT_PART', 'OGG_MEDIA', 'PT_TYPE', 'TMM_DRUG', 'AF_ROAD', 'DT_DURATION', 'TR_MEDICINE', 'TR_HUMANITIES'}

# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

 #말뭉치에 포함되어 있지 않는 태그들 추가       
labels_to_ids['CV_ACT'] = 150
ids_to_labels[150] = 'CV_ACT'

labels_to_ids['CV_STATE'] = 151
ids_to_labels[151] = 'CV_STATE'

print(ids_to_labels)
print(len(ids_to_labels))

In [ ]:
# Let's take a look at how can we preprocess the text - Take first example
text = df['text'].values.tolist()
m_len = 0
for t in text:
    if m_len < len(t):
        m_len = len(t)
        
example = text[1]

print(example)
print(m_len)

#**4. 파인튜닝한 최종 모델과 토크나이저를 불러온다.**
> ***모델, 토크나이저, epoch 입력/수정 필수!***



원하는 모델과 토크나이저를 불러오는 함수이다. \
"/content/gdrive/MyDrive/2022_lesik_workspace/lesik/model/FIXED_FINAL_EPOCH_"는 드라이브 내의 경로를 나타내는데, 왼쪽의 폴더 버튼을 눌러서 원하는 데이터의 경로를 확인할 수 있다.


In [ ]:
def load(epoch):
    model_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/model/FIXED_FINAL_EPOCH_'+ str(epoch) #모델 경로
    model = ElectraForTokenClassification.from_pretrained(model_directory, num_labels=len(labels_to_ids))
    model.to(device)
    
    tokenizer_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/tokenizer/FIXED_FINAL_EPOCH_' +str(epoch) #토크나이저 경로
    tokenizer = ElectraTokenizerFast.from_pretrained(tokenizer_directory)
    return model, tokenizer

argument로 불러오기를 원하는 epoch를 적는다.

In [ ]:
epoch = 72              #원하는 epoch로 변경
model, tokenizer = load(epoch)

#**5. 토큰화를 하기 위해 필요한 코드이다.**
> ***원하는 epoch로 수정 가능!***

epoch 개수는 고정이 아니므로 각 모델에 적절 또는 최적화 되어있는 개수로 변경하면 된다. \
(저희는 말뭉치 제외하고 72로 고정해서 학습) \
학습 중에 중단 되었을 경우, 저장된 epoch부터 이어서 학습 시킬 수 있다. 단, epochs를 저장된 epoch만큼 빼서 변경해줘야한다.
> ex.) epoch 72를 목표하였고, epoch 48까지 저장된 후 중단 되었다면 \
72-48 = 24; epochs를 24로 변경해주면 된다. 

(8.학습 실행 코드 참고)

In [ ]:
from transformers import ElectraTokenizerFast

MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 72             #원하는 epoch로 변경
LEARNING_RATE = 1e-06
MAX_GRAD_NORM = 10

In [ ]:
class ElectraDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.text[index].strip() #문장 하나
        word_labels = self.data.label[index].split() # 한 문장의 레이블들을 모아놓은 리스트

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        valid_token_list = []
        
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] == 0: #[cls]가 아니면
                continue
            valid_token_list.append(mapping)
        if len(valid_token_list) != len(word_labels): 
            print(index, len(word_labels), len(valid_token_list), sentence)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] # 워드 레이블: 문장의 레이블을 모아놓은 리스트
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        if len(labels) != 0:
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0 and mapping[1] == 0: # [cls]가 아니면
                    continue
                tok = tokenizer.convert_ids_to_tokens(encoding['input_ids'][idx]) # 토큰이 저장됨
            
                # overwrite label
                if i == len(labels):
                    break
                encoded_labels[idx] = labels[i] # 레이블 저장
                i += 1           

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['label'] = torch.as_tensor(encoded_labels)   
        return item

    def __len__(self):
        return self.len

In [ ]:
import math
train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
validation_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape)) # 0.8
print("VALIDATION Dataset: {}".format(validation_dataset.shape)) # 0.2

training_set = ElectraDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = ElectraDataset(validation_dataset, tokenizer, MAX_LEN)

FULL Dataset: (5393, 2)
TRAIN Dataset: (4314, 2)
VALIDATION Dataset: (1079, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
model.to(device) # cpu / gpu 사용 뭐 할지 선택 (꼭 필요!)

In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["label"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.0477, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 256, 152])

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter()

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

#**6. train, validation 함수를 불러오는 섹션이다.**

train을 위한 함수이다.

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['label'].to(device, dtype = torch.long)

        output = model(ids, attention_mask=mask, labels=labels)
        loss = output[0]
        tr_logits = output[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    writer.add_scalar('Train/Loss', epoch_loss, epoch)
    writer.add_scalar('Train/Accuracy', tr_accuracy, epoch)

validation을 위한 함수이다.

In [ ]:
def valid(epoch):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label'].to(device, dtype = torch.long)
            
            output = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = output[0]
            eval_logits = output[1]
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    writer.add_scalar('Validation/Loss', eval_loss, epoch)
    writer.add_scalar('Validation/Accuracy', eval_accuracy, epoch)

    return labels, predictions

#**7. save 함수를 불러오는 섹션이다.**
> ***directory/model/tokenizer 이름 변경은 필수!***

모델 저장을 위한 함수이다.

In [ ]:
import os
def save(epoch):
    directory = "/content/gdrive/MyDrive/2022_lesik_workspace/lesik/model/FTEST_EPOCH_"+ str(epoch)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save_pretrained(directory)

    torch.save(model.state_dict(), directory+"/model.pt")
    directory = "/content/gdrive/MyDrive/2022_lesik_workspace/lesik/tokenizer/FTEST_EPOCH_" + str(epoch)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    # save vocabulary of the tokenizer
    tokenizer.save_vocabulary(directory)
    tokenizer.save_pretrained(directory)
    # save the model weights and its configuration file
    print('All files saved')
    print('This tutorial is completed')

#**8. 학습을 실행하는 코드이다.**
> ***학습 과정에서 끊겼을 경우, prev_epoch 변경 필수! 그 외는 0으로 실행!***


*   prev_epoch는 학습을 시작하는 지점을 뜻하는 epoch이다.
*   학습하다 끊겼을 경우, 저장 단위의 배수를 계산하여 마지막으로 저장된 epoch로 변경 해주면 된다. \
또한, 토큰화에서도 목표하고자 하는epoch를 저장된 epoch만큼 빼서 변경해줘야 한다.
> ex.)epoch가 50에서 중단 되었을 경우, \
epoch 48까지 저장되었기 때문에 prev_epoch는 48로 시작. 토큰화의 epoch는 24로 변경. \
단, 51에서 중단되었을 경우, \
epoch 51이 저장되었다면, prev_epoch는 51부터 시작. 토큰화의 epoch는 21로 변경. \
epoch 51이 저장되지 않았다면,prev_epoch는 48부터 시작. 토큰화의 epoch는 24로 변경.
*   epoch는 현재 3의 배수로 저장되고 있으며, 변경이 가능하다.

In [ ]:
prev_epoch = 0          #학습을 시작하는 epoch
for epoch in range(prev_epoch + 1, prev_epoch + 1 + EPOCHS):
    print(f"epoch: {epoch}")
    if epoch != 0 and epoch % 3 == 0: #현재 3의 배수로 저장되고 있으며 변경 가능 (3을 변경해주면 됩니다.)
        save(epoch)
    train(epoch)
    labels, predictions = valid(epoch)

# **(선택) 모델 학습 출력**
>*** tensorboard 불러오기***

In [ ]:
writer.close()

아래 코드들을 실행하면 텐서보드를 작동시켜 그래프를 확인할 수 있다.

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir runs --port=6000